In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")
import random

In [3]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set your desired seed
set_seed(100)

In [2]:
#Notes Regarding versions:
#3.6.13 python --> 3.8.10
#0.10.0 gluonts --> 0.9.0
#Version: 1.1.5 pandas --> 1.5.3
#Version: 1.8.0 torch --> 2.1.5 --> 1.10.0 last version according to the issue answer in git
#Version: 1.19.5 numpy --> 1.23.5
#after this had issue regarding the distribution output importing changed the importing in the pts/dis_output line 34 to gluonts.torch.modules.distribution_output
# and remove it from the importing cell.
#encountered prefetch error handled it by downgrading torch version
#to install gluonts 0.9.0 had to downgrade pip version to 24.0
#!pip install optuna

In [2]:
import numpy as np
import pandas as pd
import torch
import sys
import pts.dataset
#sys.path.append(r'Thesis\Thesis\pytorch-ts-master')

In [4]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
#import gluonts.torch.distributions.distribution_output
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from pts.model.tempflow import TempFlowEstimator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
dataset = get_dataset("solar_nips", regenerate=False)
dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat_0', cardinality='137')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [7]:
# from gluonts.dataset.common import ListDataset
# train_size = int(0.8 * 7009)  # 80% of 7009 for training

# # Define train and validation datasets separately
# train_data = []
# val_data = []

# for entry in dataset.train:
#     # Create train entry
#     train_entry = entry.copy()
#     train_entry['target'] = entry['target'][:train_size]  # First 5607 values
#     train_data.append(train_entry)
    
#     # Create validation entry
#     val_entry = entry.copy()
#     val_entry['target'] = entry['target'][train_size:]  # Remaining 1402 values
#     val_data.append(val_entry)

# # Verify that train and validation sets have the expected number of time steps
# for i, (train_entry, val_entry) in enumerate(zip(train_data, val_data)):
#     train_length = len(train_entry['target'])
#     val_length = len(val_entry['target'])
    
#     print(f"Series {i}: Train length = {train_length}, Validation length = {val_length}")
    
#     # Optional: Assert that lengths are correct
#     assert train_length == 5607, f"Train length mismatch for series {i}: got {train_length}"
#     assert val_length == 1402, f"Validation length mismatch for series {i}: got {val_length}"


In [8]:
train_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))
test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))
#val_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))
# test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(train_data)), 
#                                    max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

In [9]:
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)
#dataset_val = val_grouper(val_data)

In [10]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})

In [11]:
estimator = TempFlowEstimator(
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    cell_type='GRU',
    input_size=552,
    freq=dataset.metadata.freq,
    scaling=True,
    dequantize=True,
    n_blocks=6,
    trainer=Trainer(device=device,
                    epochs=45,
                    learning_rate=0.0036476832955651584,
                    num_batches_per_epoch=100,
                    batch_size=64)
)

In [12]:
predictor = estimator.train(dataset_train, num_workers = 4)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

...

  0%|          | 0/99 [00:00<?, ?it/s]

  0%|          | 0/99 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2034819280>
Traceback (most recent call last):
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2034819280>
Traceback (most recent call last):
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is

  0%|          | 0/99 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2034819280>
Traceback (most recent call last):
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f2034819280>
Traceback (most recent call last):
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is

In [13]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 4.8067888909474945
ND: 6.366398600624585
NRMSE: 8.433601285577305
MSE: 67468.94474317798


In [14]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 5.03394814136599
ND-Sum: 6.4757982626936474
NRMSE-Sum: 7.277246419398039
MSE-Sum: 942872795.4285716


In [15]:
nlls = []
for target, forecast in zip(targets, forecasts):
    # Convert target to a numpy array if it is not already
    target_array = np.asarray(target)  # Ensure the target is a numpy array

    # Extract the target values for the prediction length
    observed = target_array[-predictor.prediction_length:]  # Shape: (prediction_length,)

    # Get forecast samples (shape: [num_samples, prediction_length, target_dim])
    samples = forecast.samples  # Shape: (100, 24, 370)

    # If multivariate, ensure dimensions match
    if samples.ndim == 3:  # (num_samples, prediction_length, target_dim)
        for dim in range(samples.shape[-1]):  # Iterate over dimensions
            dim_samples = samples[:, :, dim]  # Shape: (100, 24)
            dim_observed = observed[:, dim]   # Extract observed values for this dimension

            # Estimate likelihoods for observed values under the empirical distribution
            likelihoods = np.mean(
                np.exp(-0.5 * ((dim_observed - dim_samples) ** 2)), axis=0
            )  # Shape: (24,)
            log_likelihoods = np.log(likelihoods + 1e-10)  # Add epsilon for numerical stability

            # Compute NLL for this dimension
            nll = -np.mean(log_likelihoods)
            nlls.append(nll)
    else:  # For univariate cases
        likelihoods = np.mean(
            np.exp(-0.5 * ((observed - samples) ** 2)), axis=0
        )  # Shape: (24,)
        log_likelihoods = np.log(likelihoods + 1e-10)  # Add epsilon for numerical stability

        # Compute NLL for univariate case
        nll = -np.mean(log_likelihoods)
        nlls.append(nll)

# Aggregate NLL across all series and dimensions
average_nll = np.mean(nlls)

# Print the result
print(f"Negative Log-Likelihood (NLL): {average_nll}")


Negative Log-Likelihood (NLL): 9.716914176940918


### Solar Tuning

In [ ]:
import optuna
import torch

# Objective function for Optuna
def objective(trial):
    # Suggest values for hyperparameters
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 1e-1)
    #batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    #epochs = trial.suggest_int("epochs", 25, 75)
    n_blocks = trial.suggest_int("n_blocks", 2, 6)
    
    # Initialize the Trainer with suggested parameters
    trainer = Trainer(
        device=device,
        epochs=10,
        learning_rate=learning_rate,
        batch_size=64,
        num_batches_per_epoch=10
    )
    
    # Initialize the Estimator with suggested parameters
    estimator = TempFlowEstimator(
        target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
        prediction_length=dataset.metadata.prediction_length,
        cell_type='GRU',
        input_size=552,
        freq=dataset.metadata.freq,
        scaling=True,
        dequantize=True,
        n_blocks=n_blocks,
        trainer=trainer
    )
    
    # Train the model and get the predictor
    predictor = estimator.train(dataset_train, dataset_val, num_workers=4)
    
    # Return the validation loss for the current trial (assuming Trainer has avg_val_loss attribute)
    return trainer.avg_val_loss

# Run the Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

# Output best parameters and validation loss
print("Best parameters found:", study.best_params)
print("Best validation loss:", study.best_value)


[I 2024-11-06 01:52:00,017] A new study created in memory with name: no-name-8f92155e-2ccc-4127-8b60-3146c0b66af5


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8d844d550>
Exception ignored in: Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8d844d550>  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
Exception ignored in: 
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8d844d550><function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8d844d550>    Traceback (most recent call last):

self._shutdown_workers()  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
Traceback (most recent call last):
      File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__

self._shutdown_workers()    
  File "/home/abdul/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers

  File "/home/abdul/.lo